# Experiment to observe "Intra-CCA fairness by Reno, Cubic and BBR at Edge and Core Scale"

## Set up your FABRIC environment


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [3]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [4]:
slice = fablib.get_slice(name=slice_name)

In [5]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [6]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Generate flows

### Set experiment parameters

>delay, cca, test_duration, num_servers, flows

delay is the delay to be set at the receiver (20 ms,100 ms,200 ms)

num_servers is the number of ports to be opened on each receiver. For core scale we are opening 10 ports and for edge scale we are opening 1 port

test_duration is the time for which to send the iperf3 flows

cca is the congestion control algorithm (bbr, reno or cubic)

flows is the number of parallel flows to be send from each port

For sending 1000 flows set num_servers=10 and flows=10. This will send 100 flows from each of the 10 senders.

In [34]:
cca="reno"
delay=100
test_duration=1800
num_servers=10
flows=3

## Remove existing result files from the hosts

In [35]:
for n in receiver_nodes:
    n.execute("rm -f ./*")

for n in sender_nodes:
    n.execute("rm -f ./*")

 rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
  rm: cannot remove './examples': Is a directory
rm: cannot remove './src': Is a directory
 

In [36]:
for n in sender_nodes:
    n.execute("ls")

examples
src
examples
src
examples
src
examples
src
examples
src
examples
src
examples
src
examples
src
examples
src
examples
src


In [37]:
for n in receiver_nodes:
    n.execute("ls")

### Set delay on the receiver

> Values: 20ms, 100ms, 200ms

Now set up delay on the receiver interface:

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [38]:
for n in receiver_nodes:
    receiver_inf=n.get_interface(network_name= "link-receiver")
    receiver_inf_name = receiver_inf.get_device_name()
    n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
    n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(delay) +"ms limit 1000000")
    

### Start parallel servers on the receivers

In this, the base_port is the starting address of port number

Now start the `iperf3` flows:

In [39]:

        
base_port=50000
for i in range(0,10):
    receiver_nodes[i].execute("sudo killall iperf3")
for i in range(0,10):
    for k in range (1,num_servers+1): 
        
        server_port=base_port+k
        report_file=str(server_port)+"-server.dat"
        receiver_nodes[i].execute_thread("iperf3 -s -p "+str(server_port)+" -D --logfile "+report_file)

In [23]:
#check files in all receiver
for n in receiver_nodes:
    n.execute("ls")

50001-server.dat
50002-server.dat
50003-server.dat
50004-server.dat
50005-server.dat
50006-server.dat
50007-server.dat
50008-server.dat
50009-server.dat
50010-server.dat
50001-server.dat
50002-server.dat
50003-server.dat
50004-server.dat
50005-server.dat
50006-server.dat
50007-server.dat
50008-server.dat
50010-server.dat
50001-server.dat
50002-server.dat
50003-server.dat
50004-server.dat
50005-server.dat
50006-server.dat
50007-server.dat
50008-server.dat
50009-server.dat
50010-server.dat
50003-server.dat
50004-server.dat
50005-server.dat
50007-server.dat
50009-server.dat
50001-server.dat
50002-server.dat
50004-server.dat
50005-server.dat
50006-server.dat
50001-server.dat
50002-server.dat
50003-server.dat
50004-server.dat
50005-server.dat
50006-server.dat
50007-server.dat
50008-server.dat
50009-server.dat
50010-server.dat
50001-server.dat
50002-server.dat
50003-server.dat
50004-server.dat
50005-server.dat
50006-server.dat
50007-server.dat
50008-server.dat
50009-server.dat
50010-server.d

### Start parallel clients on the senders

In this, the base_port is the starting address of port number on the receiver


In [40]:
import time
base_port=50000


for i in range (0,10):
    sender_nodes[i].execute("sudo killall iperf3")

for i in range (0,10):
    for k in range (0,num_servers):
        server_port=base_port+k+1
        report_file="sender-10.10.2.1"+str(i)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca+".txt"
        sender_nodes[i].execute_thread("iperf3 -c 10.10.2.1"+str(i)+ " -p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca+" -P "+str(flows)+ " --format k &>"+report_file+ " &")
time.sleep(test_duration+10)              

 iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
 

In [41]:
#check files on all the senders
for n in sender_nodes:
    n.execute("ls")

examples
sender-10.10.2.10-50001-1800-reno.txt
sender-10.10.2.10-50002-1800-reno.txt
sender-10.10.2.10-50003-1800-reno.txt
sender-10.10.2.10-50004-1800-reno.txt
sender-10.10.2.10-50005-1800-reno.txt
sender-10.10.2.10-50006-1800-reno.txt
sender-10.10.2.10-50007-1800-reno.txt
sender-10.10.2.10-50008-1800-reno.txt
sender-10.10.2.10-50009-1800-reno.txt
sender-10.10.2.10-50010-1800-reno.txt
src
examples
sender-10.10.2.11-50001-1800-reno.txt
sender-10.10.2.11-50002-1800-reno.txt
sender-10.10.2.11-50003-1800-reno.txt
sender-10.10.2.11-50004-1800-reno.txt
sender-10.10.2.11-50005-1800-reno.txt
sender-10.10.2.11-50006-1800-reno.txt
sender-10.10.2.11-50007-1800-reno.txt
sender-10.10.2.11-50008-1800-reno.txt
sender-10.10.2.11-50009-1800-reno.txt
sender-10.10.2.11-50010-1800-reno.txt
src
examples
sender-10.10.2.12-50001-1800-reno.txt
sender-10.10.2.12-50002-1800-reno.txt
sender-10.10.2.12-50003-1800-reno.txt
sender-10.10.2.12-50004-1800-reno.txt
sender-10.10.2.12-50005-1800-reno.txt
sender-10.10.2.

## Analyze the results

Transfer files from hosts to router

Calculate sum of bandwidth, square of sum of bandwidth, count of flows and jfi:

In [42]:

sum_bw = []
sum_sq_bw = []
count_flow = []

for n in sender_nodes:
    (sum_sen, serr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{sum+=$7}END {print sum}'")
    sum_bw.append(float(sum_sen.strip()))
    (sum_sq, sqerr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{sum+=$7*$7}END {print sum}'")
    sum_sq_bw.append(float(sum_sq.strip()))
    (ncount, ncerr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{count+=1}END {print count}'")
    count_flow.append(int(ncount.strip()))


total_tput=sum(sum_bw)
sum_sq_tput=sum(sum_sq_bw)
count_flow=sum(count_flow)

print("Sum of bandwidth is %f Kbits/sec " % total_tput)

print("Sum of square of bandwidth is %f" % sum_sq_tput)

print("Count of flows is %d" % count_flow)

jfi= (total_tput*total_tput)/(count_flow*sum_sq_tput)

print("JFI is %f" % jfi)

93688
293506776
30
95265
303848575
30
94981
302053255
30
98590
327174698
30
95260
305340674
30
99835
334745827
30
98702
325274206
30
96612
312194606
30
97170
316144206
30
96899
313886719
30
Sum of bandwidth is 967002.000000 Kbits/sec 
Sum of square of bandwidth is 3134169542.000000
Count of flows is 300
JFI is 0.994514


In [43]:
import csv
import sys
import os

jfi_filename='jfi.csv'
if not os.path.isfile(jfi_filename):
    with open(jfi_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header ='CCA', 'Duration of Expt(sec)', 'Base RTT(ms)', 'Total Bandwidth(Kbps)', 'Sum of sq of BW', 'Flow Count', 'JFI'
        writer.writerow(header)
    
with open(jfi_filename, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    columns = cca, test_duration, delay, total_tput, sum_sq_tput, count_flow, jfi
    writer.writerow(columns)
